In [12]:
try:
# !pip install scikit-learn==0.23

    import datetime
    import os
    import pickle
    import pandas as pd
    import s3fs

    import numpy as np
    from melitk.fda import workspace

    from mktutils.google_cloud import BigQuery
    import mktutils.substatus as subs
    import os
    import base64

    import datetime
    from datetime import timedelta
    from dateutil.relativedelta import relativedelta
    from pytz import timezone

    import boto3
    s3 = boto3.client('s3')
    s3.download_file("fury-data-apps", "marketing-utils/pzenone/utils.py","utils.py")
    import sys
    sys.path.append(os.path.dirname(os.path.expanduser(".")))
    sys.path.append(os.path.dirname(os.path.expanduser("./utils.py")))
    import utils

    from causalml.inference.meta import LRSRegressor

    from utils_churn import tmp_table_q,tools_q,covariates_q,preds_q,envios_q,rates_q,presupuesto_q,cambio_q,resu_short_q,cambio2_q,inc_metrics,table_persuadidos

    if(os.environ["SECRET_DEBUG"] == "True"):
        sys.exit(0)
        
    TAKE_RATE={
        "MLA":0.095,
        "MLM":0.11,
        "MLC":0.08,
        "MCO":0.102,
        "MLB":0.148
    }

    today = datetime.datetime.now(timezone('America/Argentina/Buenos_Aires'))
    lu_day = today - timedelta(days = 9) # corre el juevs y busca el marte de la semana pasada
    lu_day = lu_day.strftime("%Y-%m-%d") # martes

    sites = ["MLA","MLM","MLB","MCO","MLC"]
    sites_str_ = "','".join(sites)
    sites_str_generico = "'" + sites_str_ + "'"

    # datetime.datetime.now(timezone('America/Argentina/Buenos_Aires'))

    teradata_user = os.environ['SECRET_TERADATA_USER_LDAP']
    teradata_pass = os.environ['SECRET_TERADATA_PASS_LDAP']

    AUTH_BIGQUERY = base64.b64decode(os.environ["SECRET_MODELLING_BIGQUERY"])
    bq = BigQuery(AUTH_BIGQUERY)

    sbs = subs.Substatus(AUTH_BIGQUERY) # suscribo en job monitor
    sbs.subscriber('start',"ALL" , "ML", "CHURN", "MIDIENDO")

    bq.execute(tmp_table_q(lu_day,sites_str_generico)).result() # temporal con sents por site

    # En caso que falle la tabla de notifications de BQ
    #     teradata_user = os.environ['SECRET_TERADATA_USER_LDAP']
    #     teradata_pass = os.environ['SECRET_TERADATA_PASS_LDAP']
    #     tera_ldap = utils.connection(teradata_user,"Clave.13",conector="teradata", type_con = "ldap")
    #     from utils_churn import temp_table_q_tera
    #     users_site = temp_table_q_tera(lu_day,"MCO","MCO_ML_PUSHML_AO_ALL_X_ACT_ALL_CHURNCUPON-12K_DEFAULT",bq,tera_ldap)

    sites_sent = bq.execute_response("select sit_site_id from meli-marketing.TEMP45.PZ_SENTS_CHURN where event_type = 'sent' group by 1 having count(1) > 1000").sit_site_id.values
    sites_str = "','".join(sites_sent)
    sites_str = "'" + sites_str + "'"
    print("Sites a medir long:", sites_str)

    if(len(sites_sent) > 0):
        tools = bq.execute_response(tools_q(sites_str)) # tool ids de churn
        tools = ",".join(tools.CAMPAIGN_ID.astype(str).values)

        bq.execute(table_persuadidos(lu_day,sites_str,tools)).result()
        covariates = bq.execute_response(covariates_q()) # genero covariables para causal inference
        print("--------------- Ready covariates")

        preds = bq.execute_response(preds_q(sites_str,str(6))) # traigo forecast 90 d
        preds = preds.rename(columns = {"user_id":"CUS_CUST_ID","sit_site_id":"SIT_SITE_ID"})
        preds.CUS_CUST_ID = preds.CUS_CUST_ID.astype(int)
        print(preds.shape)

        # puede fallar el forecast y faltar algun site, de ser asi me traigo el dia anterior
        pending_site = [x for x in sites_sent if x not in preds["SIT_SITE_ID"].unique()]

        pending_sites_str = "','".join(pending_site)
        pending_sites_str = "'" + pending_sites_str + "'"

        if(len(pending_site) > 0):
            preds2 = bq.execute_response(preds_q(pending_sites_str,str(5)))

            preds2 = preds2.rename(columns = {"user_id":"CUS_CUST_ID","sit_site_id":"SIT_SITE_ID"})
            preds2.CUS_CUST_ID = preds2.CUS_CUST_ID.astype(int)
            preds2 = preds2[["CUS_CUST_ID","SIT_SITE_ID","gmv_pred","freq_pred"]]
            preds = preds.append(preds2[["CUS_CUST_ID","SIT_SITE_ID","gmv_pred","freq_pred"]],ignore_index=True)

        pend = [x for x in sites_sent if x not in preds["SIT_SITE_ID"].unique()]
        if len(pend) > 0:
            print("Quedan sites pendientes!!: ", pend)
            raise("Faltan sites en tabla forecast")

        # Genero dataset de covariables
        preds = preds.dropna()
        users_all = pd.merge(covariates, preds[["CUS_CUST_ID","SIT_SITE_ID","gmv_pred","freq_pred"]].drop_duplicates(subset=["CUS_CUST_ID","SIT_SITE_ID"]),on=["CUS_CUST_ID","SIT_SITE_ID"])

        users_all.CUS_CUST_ID = users_all.CUS_CUST_ID.astype(int)
        users_all["freq_pred"] = users_all["freq_pred"].astype(float)
        users_all["gmv_pred"] = users_all["gmv_pred"].astype(float)
        users_all["GMV_7d"] = users_all["GMV_7d"].astype(float)
        users_all["freq_7d"] = users_all["freq_7d"].astype(float)
        users_all["GMV_MEAN"] = users_all["GMV_MEAN"].astype(float)

        users_all["recency_date"] = pd.to_datetime(users_all["recency_date"],infer_datetime_format=True,errors = "coerce")
        dia_push = users_all["recency_date"].max()
        users_all["recency_date"] = (dia_push  - users_all["recency_date"])/np.timedelta64(1, "D")
        users_all["recency"] = pd.to_datetime(users_all["recency"],infer_datetime_format=True,errors = "coerce")
        users_all["recency"] = (dia_push  - users_all["recency"])/np.timedelta64(1, "D")
        users_all["first_purchase_window"] = pd.to_datetime(users_all["first_purchase_window"],infer_datetime_format=True,errors = "coerce")
        users_all["first_purchase_window"] = (dia_push  - users_all["first_purchase_window"])/np.timedelta64(1, "D")

        users_all.recency_date = users_all.recency_date.fillna(-10)
        users_all = users_all.fillna(0)

        users_all["freq_pred"] = users_all["freq_pred"].astype(float)
        users_all["gmv_pred"] = users_all["gmv_pred"].astype(float)

        users_all["freq_pred"] = users_all[["freq_pred","freq_7d"]].sum(axis = 1)
        users_all["gmv_pred"] = users_all[["gmv_pred","GMV_7d"]].sum(axis = 1)

        resu_envios = bq.execute_response(envios_q())## Short term pasado
        resu_envios.GRUPO = resu_envios.GRUPO.replace({"sent":"TG","control":"CG"})

        rates = bq.execute_response(rates_q(lu_day,sites_str) )## Short term pasado

        presupuesto = bq.execute_response(presupuesto_q(lu_day,sites_str))
        presupuesto["AMOUNT"] = presupuesto["AMOUNT"].astype(float)
        print(presupuesto)

        cambio = bq.execute_response(cambio_q())
        cambio["CCO_TC_VALUE"] = cambio["CCO_TC_VALUE"].astype(float)

        # genero df final con resultados de largo plazo
        validos = users_all.drop_duplicates(subset=["NOTIFICATION_DATE","SIT_SITE_ID"])

        resultados_finales = pd.DataFrame([])
        for ind,line in validos.iterrows():
            print("--------",line["SIT_SITE_ID"],"--------")
            push_day = line["NOTIFICATION_DATE"]
            aux_users = users_all.loc[users_all.SIT_SITE_ID == line["SIT_SITE_ID"]]
            aux_cambio = cambio.loc[cambio.SIT_SITE_ID == line["SIT_SITE_ID"]].CCO_TC_VALUE.values[0]
            aux_presupuesto = presupuesto.loc[presupuesto.SIT_SITE_ID == line["SIT_SITE_ID"]]
            if(aux_presupuesto.shape[0] > 0): # si pincho la mkt tool ese dia
                aux_presupuesto = aux_presupuesto.AMOUNT.values[0]
            else:
                print("Sin presupuesto en SITE")
                continue

            inc_gmv,inc_revenue,inc_freq = inc_metrics(aux_users,aux_cambio,aux_presupuesto,TAKE_RATE)
            print(inc_gmv,inc_revenue,inc_freq)

            buyers_tg = resu_envios.loc[(resu_envios.GRUPO == "TG") & (resu_envios.SIT_SITE_ID == line["SIT_SITE_ID"])].buyers.values[0]
            buyers_cg = resu_envios.loc[(resu_envios.GRUPO == "CG") & (resu_envios.SIT_SITE_ID == line["SIT_SITE_ID"])].buyers.values[0]
            gasto = presupuesto.loc[presupuesto.SIT_SITE_ID == line["SIT_SITE_ID"]]["AMOUNT"].values[0]
            cambio_ = cambio.loc[cambio.SIT_SITE_ID == line["SIT_SITE_ID"]]["CCO_TC_VALUE"].values[0]
            sent_tg = rates.loc[(rates.EVENT_TYPE == "sent") & (rates.SIT_SITE_ID == line["SIT_SITE_ID"])].cant.values[0]
            sent_cg = rates.loc[(rates.EVENT_TYPE == "control") & (rates.SIT_SITE_ID == line["SIT_SITE_ID"])].cant.values[0]
            shown_ = rates.loc[(rates.EVENT_TYPE == "shown") & (rates.SIT_SITE_ID == line["SIT_SITE_ID"])].cant.values[0]
            open_ = rates.loc[(rates.EVENT_TYPE == "open") & (rates.SIT_SITE_ID == line["SIT_SITE_ID"])].cant.values[0]

            resultados = pd.DataFrame([{"BUYERS_TG":buyers_tg,"BUYERS_CG":buyers_cg,"PERSUADIDOS":users_all.loc[(users_all.SIT_SITE_ID == line["SIT_SITE_ID"]) & (users_all.GRUPO == "TG")].shape[0],"GASTO":gasto,
             "TOTALES_CG":sent_cg,"TOTALES_TG":sent_tg,"SENT":sent_tg,"SHOWN":shown_,"OPEN":open_,
             "CAMBIO":cambio_,"GMV_INC_PRED":inc_gmv,"REVENUE_INC_PRED":inc_revenue,"FREQ_INC_PRED":inc_freq,
             "SITE":line["SIT_SITE_ID"],"FECHA":push_day}])[['SITE','FECHA','BUYERS_CG', 'BUYERS_TG', 'CAMBIO', 'FREQ_INC_PRED', 'GASTO',
                                               'GMV_INC_PRED', 'OPEN', 'PERSUADIDOS', 'REVENUE_INC_PRED', 'SENT',
                                               'SHOWN', 'TOTALES_CG', 'TOTALES_TG']]

            resultados_finales = resultados_finales.append(resultados,ignore_index=True)
        resultados_finales.to_csv("churn.csv",index=False)


        bq.execute("""
            DELETE FROM meli-marketing.MODELLING.PZ_ALL_EXPERIMENTS where FECHA = DATE'"""+lu_day+"""' and sit_site_id in("""+sites_str+""") and iniciativa = 'CHURN';

            INSERT INTO meli-marketing.MODELLING.PZ_ALL_EXPERIMENTS(CUS_CUST_ID,GRUPO,FECHA,INICIATIVA,SIT_SITE_ID)
            (
                SELECT CUS_CUST_ID,NULL as GRUPO,NOTIFICATION_DATE AS FECHA,'CHURN' as INICIATIVA,SIT_SITE_ID
                FROM meli-marketing.TEMP45.PZ_TMP_PERSUADIDOS p
                WHERE GRUPO = 'TG'
            );

        """).result()

        bq.execute("DELETE FROM  meli-marketing.TEMP45.PZ_TMP_PERSUADIDOS WHERE 1=1").result()
        bq.execute("DELETE FROM  meli-marketing.TEMP45.PZ_SENTS_CHURN WHERE 1=1").result()

    else:
        pd.DataFrame([],columns = ['SITE','FECHA','BUYERS_CG', 'BUYERS_TG', 'CAMBIO', 'FREQ_INC_PRED', 'GASTO',
                                               'GMV_INC_PRED', 'OPEN', 'PERSUADIDOS', 'REVENUE_INC_PRED', 'SENT',
                                               'SHOWN', 'TOTALES_CG', 'TOTALES_TG']).to_csv("churn.csv",index=False)
        print("SIN DATOS LONG")

    print("--------------- Ready metrics long")

    ######### Short term

    lu_day = today - timedelta(days = 2) # corre los jueves el job, y busca los datos del martes
    lu_day = lu_day.strftime("%Y-%m-%d")

    sites_short = bq.execute_response("""
                select distinct sit_site_id
                from meli-marketing.MKTPUBLIC.V_BT_PUSH_NOTIFICATION_EVENT b
                WHERE SENT_DATE = DATE'"""+lu_day+"""' and b.sit_site_id in ("""+sites_str_generico+""")
                    AND lower(BATCH_ID) like '%churncupon%' and EVENT_TYPE in ('sent')
                    and APP = 'mercadolibre'
        """).sit_site_id.values

    if(len(sites_short) > 0):
        sites_str = "','".join(sites_short)
        sites_str = "'" + sites_str + "'"
        print("Sites a medir short:", sites_str)

        tools = bq.execute_response(tools_q(sites_str)) # tool ids de churn
        tools = ",".join(tools.CAMPAIGN_ID.astype(str).values)

        bq.execute(tmp_table_q(lu_day,sites_str)).result()

        # En caso que falle la tabla de notifications de BQ
    #         teradata_user = os.environ['SECRET_TERADATA_USER_LDAP']
    #         teradata_pass = os.environ['SECRET_TERADATA_PASS_LDAP']
    #         tera_ldap = utils.connection(teradata_user,teradata_pass,conector="teradata", type_con = "ldap")
    #         from utils_churn import temp_table_q_tera
    #         users_site = temp_table_q_tera(lu_day,"MCO","MCO_ML_PUSHML_AO_ALL_X_ACT_ALL_CHURNCUPON-12K_DEFAULT",bq,tera_ldap)

        resu_envio_short = bq.execute_response(resu_short_q(lu_day,sites_str,tools))

        print("--------------- Ready short queries")

        cambio = bq.execute_response(cambio2_q(lu_day,sites_str))
        cambio["CCO_TC_VALUE"] = cambio["CCO_TC_VALUE"].astype(float)

        # Genero df de corto plazo
        site_resu = pd.DataFrame([])
        for site in resu_envio_short.sit_site_id.unique():
            buyers_tg = resu_envio_short.loc[(resu_envio_short.EVENT_TYPE == "sent") & (resu_envio_short["sit_site_id"] == site)]["BUYERS"].values[0]
            buyers_cg = resu_envio_short.loc[(resu_envio_short.EVENT_TYPE == "control") & (resu_envio_short["sit_site_id"] == site)]["BUYERS"].values[0]
            tot_tg = resu_envio_short.loc[(resu_envio_short.EVENT_TYPE == "sent") & (resu_envio_short["sit_site_id"] == site)]["AMT"].values[0]
            tot_cg = resu_envio_short.loc[(resu_envio_short.EVENT_TYPE == "control") & (resu_envio_short["sit_site_id"] == site)]["AMT"].values[0]
            shown_ = resu_envio_short.loc[(resu_envio_short.EVENT_TYPE == "shown") & (resu_envio_short["sit_site_id"] == site)]["AMT"].values[0]
            open_ = resu_envio_short.loc[(resu_envio_short.EVENT_TYPE == "open") & (resu_envio_short["sit_site_id"] == site)]["AMT"].values[0]
            cambio_ = cambio.loc[cambio.SIT_SITE_ID == site]
            gasto = resu_envio_short.loc[(resu_envio_short.EVENT_TYPE == "sent") & (resu_envio_short["sit_site_id"] == site)]["AMOUNT"].values[0]

            resultados = pd.DataFrame([{"BUYERS_TG":buyers_tg,"BUYERS_CG":buyers_cg,"PERSUADIDOS":np.nan,"GASTO":gasto,
                 "TOTALES_TG":tot_tg,"TOTALES_CG":tot_cg,"SENT":tot_tg,"SHOWN":shown_,"OPEN":open_,
                 "CAMBIO":cambio_.CCO_TC_VALUE.values[0],"GMV_INC_PRED":np.nan,"REVENUE_INC_PRED":np.nan,"FREQ_INC_PRED":np.nan,
                 "SITE":site,"FECHA":lu_day}])[['SITE','FECHA','BUYERS_CG', 'BUYERS_TG', 'CAMBIO', 'FREQ_INC_PRED', 'GASTO',
                                                   'GMV_INC_PRED', 'OPEN', 'PERSUADIDOS', 'REVENUE_INC_PRED', 'SENT',
                                                   'SHOWN', 'TOTALES_CG', 'TOTALES_TG']]

            site_resu = site_resu.append(resultados,ignore_index=True)
        site_resu.to_csv("churn_daily.csv",index=False)

        print("--------------- Ready short metrics")

        ### saco cuponeros
        today = datetime.datetime.now(timezone('America/Argentina/Buenos_Aires'))

        lu_day = today - timedelta(days = 2) # corre los jueves el job, y busca los datos del martes
        fecha = lu_day.strftime("%Y-%m-%d")

        bq.execute("""

            INSERT INTO `meli-marketing.MODELLING.PZ_AUTOMATION_PUSH` (user_id, my_payload, full_payload, fecha, iniciativa)
            (
                WITH PAGOS as(
                      SELECT  pay.cus_cust_id_buy as CUS_CUST_ID,pay.SIT_SITE_ID
                      FROM  `meli-bi-data.WHOWNER.BT_MP_PAY_PAYMENTS` as pay
                      INNER JOIN `meli-bi-data.WHOWNER.BT_MKT_COUPON_V2` as cpn
                          on pay.pay_payment_id = cpn.pay_payment_id
                          and pay.cus_cust_id_buy = cpn.cus_cust_id_buy
                          AND MKT_CPN_CREATION_DATE > DATE_SUB(CURRENT_DATE() ,INTERVAL 90 DAY)
                          AND mkt_cpn_campaign_id in ("""+tools+""")
                      WHERE
                          pay.pay_status_code = 'approved'
                          AND pay.SIT_SITE_ID in ("""+sites_str+""")
                          AND pay_created_dt BETWEEN DATE'"""+fecha+"""' AND DATE_ADD(DATE'"""+fecha+"""' ,INTERVAL 1 DAY) 
                      GROUP BY 1,2
                )
                SELECT  CAST(CUS_CUST_ID as STRING) as user_id,
                CAST(NULL as STRING) as my_payload,
                CAST(NULL as STRING) as full_payload,
                DATE'"""+fecha+"""' as FECHA,
                SIT_SITE_ID || '-ML-CHURN' as INICIATIVA 
                FROM PAGOS p
                WHERE NOT EXISTS(SELECT 1 FROM `meli-marketing.MODELLING.PZ_AUTOMATION_PUSH` c 
                                 WHERE SUBSTR(c.INICIATIVA,0,3) = p.SIT_SITE_ID 
                                      AND INICIATIVA like '%-ML-CHURN'
                                      AND c.FECHA = DATE'"""+fecha+"""'
                                 )
            );
        """).result()

    else:
        pd.DataFrame([],columns = ['SITE','FECHA','BUYERS_CG', 'BUYERS_TG', 'CAMBIO', 'FREQ_INC_PRED', 'GASTO',
                                                   'GMV_INC_PRED', 'OPEN', 'PERSUADIDOS', 'REVENUE_INC_PRED', 'SENT',
                                                   'SHOWN', 'TOTALES_CG', 'TOTALES_TG']).to_csv("churn_daily.csv",index=False)
        print("SIN DATOS SHORT")

    print("--------------- Ready cuponeros bkp")

    ####### Subo a Tableau
    utils.my_download_file("s3://fury-data-apps/ltv-churn-ml/extracto.csv", "extracto.csv")
    extracto = pd.read_csv("extracto.csv", delimiter="|")

    appender = pd.read_csv("churn.csv")
    aux = pd.merge(extracto,appender[["SITE","FECHA"]].assign(dummy = 1),on=["SITE","FECHA"],how="left")
    aux = aux.loc[aux.dummy != 1].drop(["dummy"],axis = 1)

    extracto = aux.append(appender[aux.columns],ignore_index=True)

    appender = pd.read_csv("churn_daily.csv")
    aux = pd.merge(extracto,appender[["SITE","FECHA"]].assign(dummy = 1),on=["SITE","FECHA"],how="left")
    aux = aux.loc[aux.dummy != 1].drop(["dummy"],axis = 1)

    extracto = aux.append(appender[aux.columns],ignore_index=True)
    extracto.to_csv("subirme.csv",index=False,sep="|")

    utils.my_upload_file("subirme.csv","s3://fury-data-apps/ltv-churn-ml/extracto.csv")

    # Subir a bucket de BI

    sesion = boto3.session.Session()
    client = sesion.client('s3',aws_access_key_id = os.environ["SECRET_S3_PUBLIC_ACCESS"],  aws_secret_access_key = os.environ["SECRET_S3_PUBLIC_SECRET"] )
    client.upload_file("subirme.csv","bi-public-data", "pzenone/CHURN_ML/tableau/subime.csv")

    import requests
    url = 'http://internal.mercadolibre.com/bi/steps/tableau/createhyper'
    payload = {
               "user":"pzenone",
               "password":"Clave.01",
               "site":"Marketplace",
               "project":"Marketing",
               "extractName":"PZ_Churn_Metrics",
               "access":os.environ["SECRET_S3_PUBLIC_ACCESS"],
               "secret_key":os.environ["SECRET_S3_PUBLIC_SECRET"],
               "bucket":"bi-public-data",
               "delimiter":"|",
               "inputName":"pzenone/CHURN_ML/tableau/subime.csv",
               "struct":{"struct":{"SITE":"text","FECHA":"date","BUYERS_CG":"big_int","BUYERS_TG":"big_int","CAMBIO":"double",
                                  "FREQ_INC_PRED":"double","GASTO":"double","GMV_INC_PRED":"double","OPEN":"big_int",
                                  "PERSUADIDOS":"big_int","REVENUE_INC_PRED":"double","SENT":"big_int","SHOWN":"big_int",
                                  "TOTALES_CG":"big_int","TOTALES_TG":"big_int"}} 
    }

    x = requests.post(url, json = payload)
    print(x.text)

    if(x.json()["response status"] != "200"):
        raise("Error publicando en tableau")

    sbs.subscriber('end',"ALL" , "ML", "CHURN", "MIDIENDO")
    
except Exception as inst:
    print(type(inst))    
    print(inst.args)    
    print(inst)  
    
    import boto3
    s3 = boto3.client('s3')
    s3.download_file("fury-data-apps", "marketing-utils/pzenone/utils.py","utils.py")
    import utils
    
    subject = 'Falla en Midiendo Churn'
    body = 'Fallo la función de midiendo churn'
    sub = utils.Substatus("", mail_to='pedro.zenone@mercadolibre.com')
    sub.send_alert(subject, body)
    raise ValueError('Fuerzo error')

Sites a medir long: ''
SIN DATOS LONG
--------------- Ready metrics long
Sites a medir short: 'MLC','MLA','MLB','MLM'
--------------- Ready short queries
--------------- Ready short metrics
--------------- Ready cuponeros bkp
{"response status": "200", "response": "Hyper publicado en tableau con LUID: 7825f08a-3a22-4b98-9ca2-5f4ce1d1424f ", "log_id": 0, "job_name": "c466f0c4-c73e-43d1-a638-04ab5ea9305f"}

